In [1]:
# **Step 2: The Python Script (`train_snake.py`)**

import gymnasium as gym
from gymnasium import spaces
import numpy as np
import elastica as ea
from collections import defaultdict
from typing import Optional
from numpy.typing import NDArray
# from PyElastica.elastica.timestepper.symplectic_steppers import do_step
# from PyElastica.elastica.timestepper import extend_stepper_interface
from elastica.typing import RodType


from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback

import matplotlib.pyplot as plt

import os

from snake_env import FixedWavelengthContinuumSnakeEnv
import config

In [2]:
from visualization import visualization

v1 = visualization()



In [7]:
env = FixedWavelengthContinuumSnakeEnv(
    fixed_wavelength=config.ENV_CONFIG["fixed_wavelength"],
    obs_keys=config.ENV_CONFIG["obs_keys"],
)

# Configure environment parameters
env.period = config.ENV_CONFIG["period"]
env.ratio_time = config.ENV_CONFIG["ratio_time"]
env.rut_ratio = config.ENV_CONFIG["rut_ratio"]

obs, _ = env.reset()

In [8]:
model_path = os.path.join("Training", "Saved_Models", "PPO_Snake_Model")

print(f"\nLoading model from {model_path}...")
if not os.path.exists(model_path + ".zip"):
    print(f"Error: Model file not found at {model_path}.zip")
    
model = PPO.load(model_path, env=env)
print("Model loaded successfully!")



Loading model from Training/Saved_Models/PPO_Snake_Model...
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Model loaded successfully!


In [9]:
import contextlib
import io


# Run a few steps using the trained model
num_steps = 5000  # Change this to run more or fewer steps
print(f"\nRunning {num_steps} steps with trained model...")
print("=" * 70)





# Run evaluation
print(f"\nRunning {num_steps} steps with trained model...")
print("=" * 70)

total_reward = 0.0
episode_count = 0

for i in range(num_steps):
    # Get action from model
    action, _states = model.predict(obs, deterministic=True)
    # action = np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3], dtype=np.float32)
    
    # Step environment (suppress stdout from elastica)
    with contextlib.redirect_stdout(io.StringIO()):
        obs, reward, terminated, truncated, info = env.step(action)
    
    # Accumulate reward
    total_reward += reward
    
    # Print step information
    action_str = np.array2string(action, precision=6, suppress_small=True)
    status_line = (
        f"Step {i+1}/{num_steps} | "
        f"Time: {info['current_time']:.2f}s | "
        f"Forward: {info['forward_speed']:.4f} m/s | "
        f"Lateral: {info['lateral_speed']:.4f} m/s | "
        f"Reward: {reward:.4f} | "
        f"Pos: [{info['position'][0]:.3f}, {info['position'][1]:.3f}, {info['position'][2]:.3f}] | "
        f"Action: {action_str}"
    )
    print(status_line)
    
    # Check if episode ended
    if terminated or truncated:
        episode_count += 1
        print(f"\nEpisode {episode_count} ended at step {i+1}")
        print(f"  Final position: {info['position']}")
        print(f"  Episode reward: {total_reward:.4f}")
        obs, _ = env.reset()
        print("Environment reset for next episode\n")
        total_reward = 0.0

# Print summary
print("\n" + "=" * 70)
print(f"Evaluation complete!")
print(f"Total steps: {num_steps}")
print(f"Episodes completed: {episode_count}")
print(f"Average reward per step: {total_reward/num_steps:.4f}")
print("=" * 70)


Running 5000 steps with trained model...

Running 5000 steps with trained model...
Step 1/5000 | Time: 0.01s | Forward: 0.0011 m/s | Lateral: 0.0000 m/s | Reward: -0.0060 | Pos: [0.000, 0.000, 0.175] | Action: [ 0.279148  1.       -1.        1.        1.       -1.      ]
Step 2/5000 | Time: 0.02s | Forward: 0.0045 m/s | Lateral: 0.0000 m/s | Reward: -0.3375 | Pos: [-0.000, 0.000, 0.175] | Action: [ 1.       1.      -0.60963 -1.      -1.       1.     ]
Step 3/5000 | Time: 0.03s | Forward: 0.0084 m/s | Lateral: 0.0000 m/s | Reward: -0.4533 | Pos: [-0.000, 0.000, 0.175] | Action: [ 0.069546  1.        1.       -1.       -1.        1.      ]
Step 4/5000 | Time: 0.04s | Forward: 0.0061 m/s | Lateral: 0.0000 m/s | Reward: -0.4637 | Pos: [-0.000, 0.000, 0.175] | Action: [ 1.       -1.       -1.       -0.459056 -1.        1.      ]
Step 5/5000 | Time: 0.05s | Forward: 0.0064 m/s | Lateral: 0.0000 m/s | Reward: 0.0503 | Pos: [-0.000, 0.000, 0.175] | Action: [-1. -1. -1.  1. -1.  1.]
Step 6/500

In [10]:

# Visualize trajectory in 2D (X vs Z)
avg_positions = np.asarray(env.state_dict['avg_position'])
# v1.trajectory_2d(avg_positions[:, 0], avg_positions[:, 2])

# Visualize velocity norm history
avg_velocities = np.asarray(env.state_dict['avg_velocity'])
# v1.velocity_norm_history(env.state_dict['time'], avg_velocities)

# Visualize trajectory in 3D
# v1.trajectory_3d(avg_positions)

# v1.trajectory_2d(avg_positions[:, 0], avg_positions[:, 2])


# plt.plot(env.state_dict['time'], env.state_dict['torque_coeffs'])
# plt.plot(env.state_dict['time'], env.state_dict['reward'])


# Visualize current snake shape
# v1.current_shape_3d(env.shearable_rod)

# Plot video from callback data
# v1.plot_video(env.callback_data)
# v1.plot_video_limited( 
#                       env.callback_data,
#                       "video_limited1.mp4",
#                       1500
#                       )
v1.plot_video_auto_limits(
    env.callback_data,
    "video_auto_limits.mp4",
    1500,
)



Plotting limited video: using 1500 of 3283 frames, 27.28 fps, 54.98 seconds duration


100%|██████████| 1499/1499 [00:17<00:00, 87.57it/s]
